<a href="https://colab.research.google.com/github/marimsw/arparse_cmd_list/blob/main/arpparseipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
import os
import argparse
import zipfile
import pyzipper
import getpass
import requests
import urllib.parse

# Проверка и установка библиотеки pyzipper
try:
    import pyzipper
except ImportError:
    print("Установка библиотеки pyzipper...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pyzipper"])
    import pyzipper  # Импортируем библиотеку после установки


# Проверка и установка библиотеки requests
try:
    import requests
except ImportError:
    print("Установка библиотеки requests...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "requests"])
    import requests  # Импортируем библиотеку после установки

# Список папок в директории
def dataset_list(directory):
    try:
        folders = [folder for folder in os.listdir(directory) if os.path.isdir(os.path.join(directory, folder))]
        if folders:
            print(f"Folders in directory '{directory}':")
            for folder in folders:
                print(folder)
        else:
            print(f"No folders found in directory '{directory}'.")
    except FileNotFoundError:
        print(f"Directory '{directory}' not found.")
    except PermissionError:
        print(f"No access to directory '{directory}'.")


# Удаление файла
def dataset_clear(file_path):
    try:
        # Проверяем, является ли путь файлом
        if os.path.isfile(file_path):
            os.remove(file_path)  # Удаляем файл
            print(f"Файл '{file_path}' успешно удален.")
        else:
            print(f"'{file_path}' не является файлом или не существует.")
    except FileNotFoundError:
        print(f"Ошибка: Файл '{file_path}' не найден.")
    except PermissionError:
        print(f"Ошибка: У вас нет прав для удаления файла '{file_path}'.")
    except Exception as e:
        print(f"Произошла ошибка при удалении файла '{file_path}': {e}")


# Распаковка ZIP-архива
def unzip_file(zip_file_path, extract_to):
    try:
        # Проверяем, существует ли ZIP-файл
        if not os.path.exists(zip_file_path):
            raise FileNotFoundError(f"Файл {zip_file_path} не найден.")

        # Проверяем, является ли файл ZIP-архивом
        if not zipfile.is_zipfile(zip_file_path):
            raise zipfile.BadZipFile(f"Файл {zip_file_path} не является ZIP-архивом.")

        # Запрашиваем пароль у пользователя
        password = getpass.getpass("Введите пароль для ZIP-архива(если есть): ")

        # Открываем ZIP-архив
        with pyzipper.AESZipFile(zip_file_path, 'r') as zip_ref:
            # Устанавливаем пароль
            zip_ref.pwd = password.encode('utf-8')

            # Извлекаем все содержимое
            zip_ref.extractall(extract_to)
            print(f"Файлы успешно извлечены в {extract_to}")

    except FileNotFoundError as e:
        print(f"Ошибка: {e}")
    except zipfile.BadZipFile as e:
        print(f"Ошибка: {e}")
    except RuntimeError as e:
        if 'Bad password' in str(e):
            print("Ошибка: Неверный пароль для ZIP-архива.")
        else:
            print(f"Ошибка: {e}")
    except PermissionError:
        print("Ошибка: У вас нет разрешения на запись в указанную директорию.")
    except Exception as e:
        print(f"Произошла непредвиденная ошибка: {e}")


# Скачивание ZIP-архива по ссылке с S3
def download_archive(url, file_name, save_path):
    try:
        # Кодирование ссылки для правильной обработки символов и русских букв
        encoded_url = urllib.parse.quote(url, safe=':/?=&%')
        response = requests.get(encoded_url, stream=True)
        response.raise_for_status()  # Проверяем, что запрос был успешным
        full_path = os.path.join(save_path, file_name)
        if os.path.exists(full_path):
            print("Файл с таким именем уже существует. Хотите перезаписать его? (y/n)")
            answer = input().lower()
            if answer != 'y':
                print("Скачивание отменено.")
                return
        with open(full_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
        print(f"Архив скачан успешно: {full_path}")
    except requests.exceptions.RequestException as e:
        print(f"Ошибка запроса: {e}")

def main():
    parser = argparse.ArgumentParser(description='Утилита для работы с файлами и папками')
    subparsers = parser.add_subparsers(dest='command')

    list_parser = subparsers.add_parser('list', help='Список папок в директории')
    list_parser.add_argument('directory', help='Путь к директории')

    clear_parser = subparsers.add_parser('clear', help='Удаление файла')
    clear_parser.add_argument('file_path', help='Путь к файлу')

    unzip_parser = subparsers.add_parser('unzip', help='Распаковка ZIP-архива')
    unzip_parser.add_argument('zip_file_path', help='Путь к ZIP-файлу')
    unzip_parser.add_argument('extract_to', help='Директория для извлечения')

    download_parser = subparsers.add_parser('download', help='Скачивание ZIP-архива')
    download_parser.add_argument('url', help='Ссылка на ZIP-архив')
    download_parser.add_argument('file_name', help='Имя файла для сохранения архива')
    download_parser.add_argument('save_path', help='Путь для сохранения архива')

    args = parser.parse_args()

    if args.command == 'list':
        dataset_list(args.directory)
    elif args.command == 'clear':
        dataset_clear(args.file_path)
    # elif args.command == 'unzip':
    #     unzip_file(args.zip_file_path, args.extract_to)
    elif args.command == 'download':
        download_archive(args.url, args.file_name, args.save_path)
    else:
        parser.print_help()

if __name__ == '__main__':
    main()